# Guided Project - Implementing A Key-Value Database

[<b>Solution</b>](https://github.com/dataquestio/solutions/blob/master/Mission234Solutions.ipynb)

In this guided project, we will be using the b-tree data structure from before as the building block for a fully functioning, saving-to-disk, <b>key-value store</b>. A [key-value store](https://en.wikipedia.org/wiki/Key%E2%80%93value_database) is a database that operates similar to a Python dictionary but stored in a file. There are the important <code>get</code> and <code>set</code> methods that the dictionary contains, but the key-value store will also provide an API for saving to disk, loading from disk, range queries of our data, and others. Our goal will be to create an easy to use, flexible, and adaptable key value store that other developers could use in their projects.

For this guided project, we will inherit the majority of the behavior from the <code>BTree</code> class. However, we will redesign some of the API to make it more user friendly. Finally, we will be implementing some additional behavior to make it function as the key-value database that we expect.

## 1. Creating a Key-Value Store Class

Every key-value store (abbreviated to kv store) implements a <code>get()</code> and <code>set()</code> method that serve as its most important functionality. Like a Python <code>dict</code>, the kv store will <code>set</code> a <code>value</code> for some given <code>key</code> only if value is not <code>None</code>. However, the <code>set()</code> method should also function different from a Python dictionary by requiring the uniqueness of the <code>key</code>, that means if the <code>key</code> exists in the kv store, then throw a duplicate error.

For the <code>get()</code> method, we also want to replicate the same behavior from the Python dictionary. If we wish to <code>get()</code> the value for some given key, then we should get the saved value. However, if that key does not exist, the key-value store should throw an exception.

Remember to think about the API design as you develop the <code>get()</code> and <code>set()</code> methods. A handy class that you might want to use is the <code>NodeKey</code> class that we have implemented before. With this class, try to make it so the user of the kv store does not have to create a <code>NodeKey</code> objects first before calling <code>set()</code> on the kv store.

## 2. Verify Type of Key and Setting <code>\_\_init\_\_()</code> of Parent Class

Right now we allow any type to be set as our key. For our kv store to work with a b-tree, we need to set some data type as our key. If we don't then we would be doing comparisons on <code>ints</code> and <code>strs</code> for example. We should override the initializer and set some <code>type</code> that we use to validate the inserted keys in our <code>set()</code> method.

An important thing to note is that when overriding the <code>\_\_init\_\_()</code> method, we also want to also call the parent <code>\_\_init\_\_()</code> method. If you don't then we would not be able to set the <code>self.t</code> and <code>self.root</code> properties that are expected. To call the parent <code>\_\_init\_\_()</code> method, we use a function named <code>super()</code> which is just another way of writing the <code>BTree</code> class.

## 3. New Range Query Method

The current range queries for <code>greater_than()</code> and <code>less_than()</code> are a bit tedious to work with. First, you have to pass in the root node, then you have to pass in a value, an optional upper bound, and then an inclusive! All these can be frustrating for a user who just wants to do a simple range query on their data.

To solve this user experience problem, we will implement a new range query method that will abstract away the tedious work for the user. We will still use the old range queries under the hood, but we provide the user with better error returns so they can iteratively learn how to use the range query.

The user should not have to pass in the root node, it's too tedious to remember every time.

## 4. Dump and Load the KV Store

Now it's time to get to the bread and butter of the kv store, saving and loading it to disk. The behavior we want to provide is to let a user pass in a filename to load or save to the object to disk. To make it easier for the user, we will attach a our filetype name to the end and check if that file is available to read or write out to.

However, one thing to keep in mind is that we want to provide a <code>load()</code> function that is not an instance method of the <code>DQKV</code> class. The reason being is that we want to load from a file to create a <code>DQKV</code> instance. If it was an instance method, then it would be defeating the purpose of the behavior it is trying to provide.

## 5. Load from Dictonary

Write a <code>load_from_dict()</code> method that loads in keys and values from a Python <code>dict</code>.

In [1]:
from btree_DataQuest import NodeKey, Node, BTree
import pickle

### Creating DQKV class

In [2]:
class DQKV(BTree):
    def __init__(self, type_):
        self.type = type_
        super().__init__(3) # Sets t (degree) of BTree
    
    def set(self, key, value):
        """
        Sets NodeKey in BTree Structure
        """
        
        # Check for failing conditions
        if value == None:
            raise ValueError('value can not be None')
        if not isinstance(key, self.type):
            raise KeyError('key must be type {}'.format(self.type))
        if self.search(self.root, key) != None:
            raise ValueError('key must be unique')
            
        self.insert(NodeKey(key,value))
        
    def get(self, key):
        """
        Gets value with given key
        """
        result = self.search(self.root, key)
        
        if result == None:
            raise Exception('key "{}" not in key-value database'.format(key))
        else:
            return result
    
    def range_query(self, l, inclusive=False):
        """
        Return keys within provided list
        l: list with two values, lower and upper
        """
        
        # Check for failing conditions
        if len(l) != 2:
            raise TypeError('l needs two values, {} were given'.format(len(l)))
        for v in l:
            if isinstance(v, self.type) or isinstance(v, type(None)):
                pass
            else:
                raise TypeError('l values must be {} or {}'.format(self.type,type(None)))
        
        lower = l[0]
        upper = l[1]
        
        if upper == None:
            return self.greater_than(self.root, lower, inclusive=inclusive)
        if lower == None:
            return self.less_than(self.root, upper, inclusive=inclusive)
        
        return self.greater_than(self.root, lower, upper_bound=upper, inclusive=inclusive)
    
    def dump(self, file):
        """
        Save Key-Value data to file on disk
        """

        # Check for failing conditions
        if not isinstance(file, str):
            raise TypeError('file names needs to be {}'.format(type(str)))
        
        ending = '.dqdb'
        f_name = file + ending
        
        with open(f_name, 'wb') as f:
            pickle.dump(self,f)
            
    def load_from_dict(self, d):
        """
        Load data from a dictonary
        """
        
        for k,v in d.items():
            self.set(k,v)
        
def load_dqkv(file):
    """
    Load Key-Value data from disk
    """
    
    ending = '.dqdb'
    f_name = file + ending
    
    with open(f_name, 'rb') as f:
        return pickle.load(f)

#### Testing Cases Point 1. Creating a Key-Value Store Class

In [3]:
dqkv = DQKV(int)
dqkv.set(5,7)
dqkv.set(6,12)
dqkv.set(8,9)
dqkv.set(9,1)
dqkv.set(10,15)
print(dqkv.root)
print(dqkv.root.children)

<Node: [<NodeKey: (5, 7)>, <NodeKey: (6, 12)>, <NodeKey: (8, 9)>, <NodeKey: (9, 1)>, <NodeKey: (10, 15)>]>
[]


In [4]:
# Value == None Exception
dqkv.set(7, None)

ValueError: value can not be None

In [5]:
# Key Can not be in database
dqkv.set(5,12)

ValueError: key must be unique

In [6]:
# Key not found exception
dqkv.get(47)

Exception: key "47" not in key-value database

#### Testing Cases Point 2. Verify Type of Key and Setting <code>\_\_init\_\_()</code> of Parent Class

In [7]:
# Type Error
dqkv.set('text',47)

KeyError: "key must be type <class 'int'>"

#### Testing Cases Point 3. New Range Query Method

In [8]:
dqkv.range_query([5.0,None])

TypeError: l values must be <class 'int'> or <class 'NoneType'>

In [9]:
dqkv.range_query([0])

TypeError: l needs two values, 1 were given

In [10]:
dqkv.range_query([0,None])

[<NodeKey: (5, 7)>,
 <NodeKey: (6, 12)>,
 <NodeKey: (8, 9)>,
 <NodeKey: (9, 1)>,
 <NodeKey: (10, 15)>]

#### Testing Cases Point 4. Dump and Load the KV Store

In [11]:
dqkv.dump('test_1')

In [12]:
return_1 = load_dqkv('test_1')

In [13]:
return_1.root

<Node: [<NodeKey: (5, 7)>, <NodeKey: (6, 12)>, <NodeKey: (8, 9)>, <NodeKey: (9, 1)>, <NodeKey: (10, 15)>]>

#### Testing Case Point 5. Load From Dictonary

In [14]:
dqkv.load_from_dict({101: "test_101",102:"test_102"})

In [15]:
print(dqkv.root)
print(dqkv.root.children)

<Node: [<NodeKey: (8, 9)>]>
[<Node: [<NodeKey: (5, 7)>, <NodeKey: (6, 12)>]>, <Node: [<NodeKey: (9, 1)>, <NodeKey: (10, 15)>, <NodeKey: (101, test_101)>, <NodeKey: (102, test_102)>]>]
